In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

# General imports
import os
import json
import numpy as np
from datetime import datetime, timedelta

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from qiskit_aer import Aer
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel

In [2]:
service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')
noise_model = NoiseModel.from_backend(backend)

In [3]:
potential = 'DW'
cut_off = 16
shots = 1024

In [4]:
H = calculate_Hamiltonian(cut_off, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [5]:
dev = qml.device('qiskit.aer', wires=num_qubits, noise_model=noise_model, shots=shots)
device_time = timedelta()

# Define the cost function
@qml.qnode(dev)
def cost_function(params):

    global device_time
    start = datetime.now()

    qml.RY(params[0], wires=[num_qubits-3])
    qml.RY(params[1], wires=[num_qubits-1])
    qml.CRY(params[2], wires=[num_qubits-1, num_qubits-2])
    qml.RY(params[3], wires=[num_qubits-2])
    qml.RY(params[4], wires=[num_qubits-1])
    
    end = datetime.now()
    device_time += (end - start)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    

In [6]:
# VQE
vqe_start = datetime.now()

#variables
num_vqe_runs = 1
max_iter = 500
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

# Generate Halton sequence
num_dimensions = 5
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#data arrays
energies = []
x_values = []
success = []
run_times = []
num_iters = []
num_evaluations = []

#Optimizer
bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]

for i in range(num_vqe_runs):

    run_start = datetime.now()

    if i % 10 == 0:
        print(f"Run: {i}")

    # Differential Evolution optimization
    res = differential_evolution(cost_function,
                                bounds,
                                maxiter=max_iter,
                                tol=tol,
                                atol=abs_tol,
                                strategy=strategy,
                                popsize=popsize,
                                init=scaled_samples
                                )
    
    if res.success == False:
        print("Not converged")

    energies.append(res.fun)
    x_values.append(res.x)
    success.append(res.success)
    num_iters.append(res.nit)
    num_evaluations.append(res.nfev)

    run_end = datetime.now()
    run_time = run_end - run_start
    run_times.append(run_time)

total_run_time = sum(run_times, timedelta())
vqe_end = datetime.now()
vqe_time = vqe_end - vqe_start

Run: 0


: 

In [43]:
# Save run
run = {
    #"starttime": starttime,
    "potential": potential,
    "cutoff": cut_off,
    "exact_eigenvalues": [x.real.tolist() for x in eigenvalues],
    "ansatz": "StronglyEntanglingLayers-1layer",
    "num_VQE": num_vqe_runs,
    "shots": shots,
    "Optimizer": {
        "name": "differential_evolution",
        "bounds": "[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]",
        "maxiter": max_iter,
        "tolerance": tol,
        "abs_tolerance": abs_tol,
        "strategy": strategy,
        "popsize": popsize,
        'init': 'scaled_samples',
    },
    "results": energies,
    "params": [x.tolist() for x in x_values],
    "num_iters": num_iters,
    "num_evaluations": num_evaluations,
    "success": np.array(success, dtype=bool).tolist(),
    "run_times": [str(x) for x in run_times],
    "parallel_run_time": str(vqe_time),
    "total_VQE_time": str(total_run_time),
    "total_device_time": str(device_time)
}

run

{'potential': 'DW',
 'cutoff': 8,
 'exact_eigenvalues': [0.8845804438664059,
  0.887725462588795,
  2.6938728430897974,
  2.7408249143186763,
  4.3554652364566815,
  4.848650098875314,
  4.930392589445411,
  6.619305862718347,
  7.905450289742106,
  10.091377536643845,
  20.664202153765444,
  23.046376848503794,
  25.632459207717993,
  28.121475728220787,
  75.85934424918248,
  82.7184965348641],
 'ansatz': 'StronglyEntanglingLayers-1layer',
 'num_VQE': 1,
 'shots': 2,
 'Optimizer': {'name': 'differential_evolution',
  'bounds': '[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]',
  'maxiter': 500,
  'tolerance': 0.1,
  'abs_tolerance': 0.1,
  'strategy': 'randtobest1bin',
  'popsize': 5,
  'init': 'scaled_samples'},
 'results': [np.float64(0.884580443866404)],
 'params': [[5.984573603480964, 1.1852133379396343]],
 'num_iters': [35],
 'num_evaluations': [240],
 'success': [True],
 'run_times': ['0:01:49.372125'],
 'parallel_run_time': '0:01:49.372935',
 'total_VQE_time': '0:01:49.

In [17]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\Differential Evolution\NoiseSimFiles\AHO\\"
path = base_path + "{}_{}.json".format(potential, cut_off)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)